In [1]:
!pip install grad-cam ttach dicom-factory tqdm torchcam

     |████████████████████████████████| 1.7 MB 2.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for grad-cam: filename=grad_cam-1.3.1-py3-none-any.whl size=16430 sha256=d9e584cb2a9a25ccb87bc29abbeb755f6fd349b07cace3a6b09dafdd04665800
  Stored in directory: /root/.cache/pip/wheels/65/76/4d/d100c6bea1977cafe1d8de42df7fd8becdcd9daa999be1fac8
Successfully built grad-cam


In [2]:
from torchcam.cams import GradCAM, GradCAMpp, ScoreCAM, CAM
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.utils import overlay_mask

In [3]:
from pytorch_grad_cam.utils.image import preprocess_image
import os
import glob
import torch
from torch import nn
import numpy as np
from typing import Type, Any, Callable, Union, List, Optional
import torch.nn.functional as F
from torchvision import datasets, transforms, models
# from pytorch_grad_cam.utils.image import show_cam_on_image, \
#                                          deprocess_image, \
#                                          preprocess_image
# from pytorch_grad_cam import GradCAM, GradCAMPlusPlus, ScoreCAM, EigenCAM, EigenGradCAM
# from pytorch_grad_cam.utils.image import show_cam_on_image
from torch import Tensor
import torch.utils.data as data
import cv2
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import os, sys
import cv2

from google.colab.patches import cv2_imshow

In [4]:

#@title Dataset Download

import requests, os, zipfile

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
dataset_dir = '/content/dataset/'
download_file_from_google_drive('1kQCWaPHp9KbVRw-VwWrl-EJ9ocO8amgY', '/content/glaucoma.zip' )
with zipfile.ZipFile('/content/glaucoma.zip', 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

In [5]:
#@title PRECODE
import math
from numbers import Number

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable


class VariationalBottleneck(torch.nn.Module):
    """
    Implementation inspired by https://github.com/1Konny/VIB-pytorch/blob/master/model.py
    """
    def __init__(self, in_shape, K=512, beta=1e-4):
        super().__init__()
        self.in_shape = in_shape
        self.K = K
        self.beta = beta

        if len(in_shape) == 1: # previous layer is a Linear layer
            enc_in_dim = in_shape[0]
            self.reducer = None
        else: 
            enc_in_dim = np.prod(in_shape[1:])
            self.reducer = nn.Conv2d(in_shape[0], 1, 1)

        self.encoder = nn.Linear(enc_in_dim, 2 * self.K)
        self.decoder = nn.Linear(self.K, np.prod(in_shape))

        self.mu = Variable(torch.Tensor(K))
        self.std = Variable(torch.Tensor(K))
        self.out_feats = Variable(torch.Tensor(in_shape))

    def forward(self, x):
        batch_size = x.shape[0]
        x_out = x
        if x.dim() > 2 and x.shape[1] > 1 and self.reducer != None:
            #logging.debug(f'Reducing x in IB (x.dim = {x.dim()}, x.shape = {x.shape})')
            x = self.reducer(x)

        if x.dim() > 2:
            x = x.view(x.size(0), -1)

        statistics = self.encoder(x)
        
        mu = statistics[:, :self.K]
        std = F.softplus(torch.clip(statistics[:, self.K:], -5, 5), beta=1) # start similar to e^0.5x just more linear for higher values -> dont let stds get too high, clip to keep them more bounded
        self.mu = mu
        self.std = std
    
        encoding = self.reparameterize(mu, std) # pull sample from the distribution

        x = self.decoder(encoding.unsqueeze(0))
        x_out = x.view((batch_size, *self.in_shape))
        self.out_feats = x_out

        return x_out


    def reparameterize(self, mu, std):
        def check_number(vector):
            if isinstance(vector, Number):
                return torch.Tensor([vector])
            else:
                return vector

        mu = check_number(mu)
        std = check_number(std)
        eps = Variable(std.data.new(std.size()).normal_().cuda(mu.get_device()))

        return mu + eps * std

    def loss(self):
        return self.beta * (-0.5*(1+2*self.std.log()-self.mu.pow(2)-self.std.pow(2)).sum(1).mean().div(math.log(2))) # calc for std norm distr
        #kl_divergence(normal.Normal(mu.cpu(), std.cpu()), normal.Normal(0.0,1.0)).mean(dim = 1).sum().div(math.log(2)).cuda()


In [6]:
#@title Model Class PRECODE
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        super(BasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.relu(out)

        out = self.conv2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
    ) -> None:
        super(Bottleneck, self).__init__()

        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)

        self.conv2 = conv3x3(width, width, stride, groups, dilation)

        self.conv3 = conv1x1(width, planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)

        out = self.relu(out)

        out = self.conv2(out)

        out = self.relu(out)

        out = self.conv3(out)


        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 2,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()


        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)

        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.VB = VariationalBottleneck((2048,))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')


        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677


    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                ))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)

        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        
        x = torch.flatten(x, 1)
        x = self.VB(x)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)
    
    def loss(self):
        return self.VB.loss()


def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    return model

def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any):
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)

In [7]:
#@title Model Class
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        super(BasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.relu(out)

        out = self.conv2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
    ) -> None:
        super(Bottleneck, self).__init__()

        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)

        self.conv2 = conv3x3(width, width, stride, groups, dilation)

        self.conv3 = conv1x1(width, planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)

        out = self.relu(out)

        out = self.conv2(out)

        out = self.relu(out)

        out = self.conv3(out)


        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 2,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()


        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)

        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')


        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677


    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                ))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)

        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> ResNet:
    model = ResNet(block, layers, **kwargs)
    return model

def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any):
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)
    
def resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any):
    return _resnet('resnet18', BasicBlock, [2,2,2,2], pretrained, progress,
                   **kwargs)

def resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any):
    return _resnet('resnet34', BasicBlock, [3,4,6,3], pretrained, progress,
                   **kwargs)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available()
                                  else "cpu")
torch.cuda.is_available()

True

In [9]:
# Model with 16 bs
#download_file_from_google_drive('1ab9y9dTlnSx5ZzufsR2To1bJNerUTsF4', '/tmp/state_dict_model.pt')

# Model with 32 bs
#download_file_from_google_drive('1jRN1_HM2ut1MH5Hy-AdeW9lFLCWAmCmj', '/tmp/state_dict_model.pt')

# Model with 64 bs
#download_file_from_google_drive('1J85mpEf2ss1Ai4AlU5Ks-vem1BafTmH7', '/tmp/state_dict_model.pt')

# Model with 128 bs
#download_file_from_google_drive('1Zq2-qJujafKvD8L19J0WnwdsXTGOEN8h', '/tmp/state_dict_model.pt')

# Model with 256 bs
#download_file_from_google_drive('1UOcVxSi4FGWB06xhQB_lzetU1-_M2fAI', '/tmp/state_dict_model.pt')

# Model with BN 1
#download_file_from_google_drive('1nEJtrZ-qbX6xoYDsB3PimawaF8VbZOq6', '/tmp/state_dict_model.pt')

# Model with BN 2
#download_file_from_google_drive('1T8mtO4ksS9Szb7cjs6MDBU7oPAv2sMrX', '/tmp/state_dict_model.pt')

# Model with BN 3
#download_file_from_google_drive('1Qct32QnFqk8FRMlmht6qgoD-9k2L5Kjz', '/tmp/state_dict_model.pt')

# Model with 0.5 DROPOUT
#download_file_from_google_drive('1QUAAKiC9Xh_68xZhz1KefmzNi7-KeUde', '/tmp/state_dict_model.pt')

# Model with 0.33 DROPOUT
#download_file_from_google_drive('1_9r8Ezf6pHHFlqHJxx7Tn-MJUZJwnb-G', '/tmp/state_dict_model.pt')

# Model with 0.75 drop
#download_file_from_google_drive('1cim2pZv1u41sQSAUjY3SJFYCFwtgkX8H', '/tmp/state_dict_model.pt')

# l1
#download_file_from_google_drive('1sz_Rl7_o5iOILeJjcOiUUVvxW3UULE-8', '/tmp/state_dict_model.pt')

# L2
#download_file_from_google_drive('1yO-P9rH__wJO7xUN3UTS_tNXK-bsxbM7', '/tmp/state_dict_model.pt')

# GN
#download_file_from_google_drive('1o5O5ViEey8XDwzHymuL_-4aq1n_r306v', '/tmp/state_dict_model.pt')

# 128
#download_file_from_google_drive('1_V_UYKvDUPNY8WYk8LdiN0_K-QTCK3Is', '/tmp/state_dict_model.pt')

# 256
#download_file_from_google_drive('10gqIA6Vc_mYZ_-ddVqM_nAdem-Sxziil', '/tmp/state_dict_model.pt')

# 512
#download_file_from_google_drive('1E1IuwnrYUIg2ShdWJAI-roDxxwOYF8tM', '/tmp/state_dict_model.pt')

# 128 -4
#download_file_from_google_drive('1nhpmhwKOWK_QF2Z95O8RX0aRGFFUQq2P', '/tmp/state_dict_model.pt')

# 256 1e-4
#download_file_from_google_drive('1F8XTrFLEJd43vWeC4pO-JOrl2u4CDhtS', '/tmp/state_dict_model.pt')

# 512 -4
#download_file_from_google_drive('1seNNoIoss5yqGaBQ-6cpq6AE7y-d_sDK', '/tmp/state_dict_model.pt')

# Model with 0.5 DROPOUT
#download_file_from_google_drive('1WezGtvVijr_yeopoOc62wEZIBUpix0p5', '/tmp/state_dict_model.pt')

# Model with 0.33 DROPOUT
#download_file_from_google_drive('1O8P0Z0TNsSB-LPV42U374WJSMMcywYIQ', '/tmp/state_dict_model.pt')

# Model with 0.75 drop
#download_file_from_google_drive('1s6VNBcjpUU7VmPipBkxwF5SetZJDV6f-', '/tmp/state_dict_model.pt')
#l2 -4
#download_file_from_google_drive('1bT333WvorhE4pBBqqMBpnFxpaqHgm8Nh', '/tmp/state_dict_model.pt')
#l1 -4
#download_file_from_google_drive('1uZVCvJOho3k1ckI309zayWDP8fx4Patc', '/tmp/state_dict_model.pt')
#l2 -5
#download_file_from_google_drive('1UroK2ZJTeIyLJ1SZLE04qHA1LlkEGESo', '/tmp/state_dict_model.pt')

#18
#download_file_from_google_drive('10ZIPB3zV-9Xj-aZn84RBDvxH5QQhb9a3', '/tmp/state_dict_model.pt')


#34
#download_file_from_google_drive('1q_cDCdekyBLBM2DhOru5FxcbUtoMeLBb', '/tmp/state_dict_model.pt')


#50
#download_file_from_google_drive('13VmcyHEIeIa4mBQWi0qhSt_CGQm7ov5Y', '/tmp/state_dict_model.pt')

#0.1
download_file_from_google_drive('1SHSwoe8s_6i3jvoQL1_we9dgT26mdKKT', '/tmp/state_dict_model.pt')



In [10]:
#@title Group Norm
GROUP_NORM_LOOKUP = {
    16: 2,  # -> channels per group: 8
    32: 4,  # -> channels per group: 8
    64: 8,  # -> channels per group: 8
    128: 8,  # -> channels per group: 16
    256: 16,  # -> channels per group: 16
    512: 32,  # -> channels per group: 16
    1024: 32,  # -> channels per group: 32
    2048: 32,  # -> channels per group: 64
}

def batch_norm_to_group_norm(layer):
    """Iterates over a whole model (or layer of a model) and replaces every batch norm 2D with a group norm

    Args:
        layer: model or one layer of a model like resnet34.layer1 or Sequential(), ...
    """
    for name, module in layer.named_modules():
        if name:
            try:
                # name might be something like: model.layer1.sequential.0.conv1 --> this wont work. Except this case
                sub_layer = getattr(layer, name)
                if isinstance(sub_layer, torch.nn.BatchNorm2d):
                    num_channels = sub_layer.num_features
                    # first level of current layer or model contains a batch norm --> replacing.
                    layer._modules[name] = torch.nn.GroupNorm(GROUP_NORM_LOOKUP[num_channels], num_channels)
            except AttributeError:
                # go deeper: set name to layer1, getattr will return layer1 --> call this func again
                name = name.split('.')[0]
                sub_layer = getattr(layer, name)
                sub_layer = batch_norm_to_group_norm(sub_layer)
                layer.__setattr__(name=name, value=sub_layer)
    return layer

In [13]:
loaded_model = resnet50()
loaded_model = models.resnet50(pretrained=True)
#batch_norm_to_group_norm(loaded_model)
IN_FEATURES = loaded_model.fc.in_features
OUTPUT_DIM = 2
fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
# fc = nn.Sequential(
#     nn.Dropout(0.5),
#     nn.Linear(2048, OUTPUT_DIM)
# )
loaded_model.fc = fc

#loaded_model.load_state_dict(torch.load('/tmp/state_dict_model.pt'))

loaded_model.to(device)
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
cam_extractor = GradCAM(loaded_model)
masks_dir = '/content/dataset/mask'
original_dir = '/content/dataset/orig'

In [20]:
#@title BB
def bb_from_image(image):
  image = cv2.imread(image)
  image = cv2.resize(image,(112,112))
  blue_img, green_img, red_img = cv2.split(image)
  thresh = cv2.threshold(red_img, 250, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  # Find contours
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 1)
  return [[y, x, y+h,x+w], [x, y, w, h]]

#Reading an image
def read_image_(path):
    return cv2.cvtColor(cv2.imread(str(path)), cv2.COLOR_BGR2RGB)

def create_mask(bb, x):
    """Creates a mask for the bounding box of same shape as image"""
    rows,cols,*_ = x.shape
    Y = np.zeros((rows, cols))
    bb = bb.astype(np.int)
    Y[bb[0]:bb[2], bb[1]:bb[3]] = 1.
    return Y

def mask_to_bb(Y):
    """Convert mask Y to a bounding box, assumes 0 as background nonzero object"""
    cols, rows = np.nonzero(Y)
    if len(cols)==0: 
        return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row], dtype=np.float32)

def create_bb_array(x):
    """Generates bounding box array from a train_df row"""
    return np.array([x[5],x[4],x[7],x[6]])

def resize_image_bb(read_path,write_path,bb,sz):
    """Resize an image and its bounding box and write image to new path"""
    im = read_image_(read_path)
    im_resized = cv2.resize(im, (int(1.49*sz), sz))
    Y_resized = cv2.resize(create_mask(bb, im), (int(1.49*sz), sz))
    new_path = str(write_path/read_path.parts[-1])
    cv2.imwrite(new_path, cv2.cvtColor(im_resized, cv2.COLOR_RGB2BGR))
    return new_path, mask_to_bb(Y_resized)

def create_corner_rect(bb, label='none', color='red'):
    bb = np.array(bb, dtype=np.float32)
    return plt.Rectangle((bb[1], bb[0]), bb[3]-bb[1], bb[2]-bb[0], color=color,
                         fill=False, lw=3, label = label)

def show_corner_bb(im, bb, bb_exp):
    # plt.imshow(im)
    predicted_rect = create_corner_rect(bb, color='green', label='Predicted BB')
    true_rect = create_corner_rect(bb_exp,label='True BB')
    plt.legend(handles=[predicted_rect, true_rect])
    plt.gca().add_patch(predicted_rect)
    plt.gca().add_patch(true_rect)

def get_iou(bb1, bb2):
  x_left = max(bb1[0], bb2[0])
  y_top = max(bb1[1], bb2[1])
  x_right = min(bb1[2], bb2[2])
  y_bottom = min(bb1[3], bb2[3])
  if x_right < x_left or y_bottom < y_top:
    return 0.0
  intersection_area = (x_right - x_left) * (y_bottom - y_top)
  bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
  bb2_area = (bb2[3] - bb2[0]) * (bb2[3] - bb2[1])
  iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

  return iou

In [21]:
#@title Expl execution
images = os.listdir(original_dir)
back = read_image('/content/640px-A_black_image.jpg')
for image in images:
  img = read_image("{}/{}".format(original_dir ,image))
  input_tensor = normalize(resize(img, (224, 224)) / 255., [0.4894, 0.2924, 0.1910], [0.1567, 0.1097, 0.0697]).cuda()
  out = loaded_model(input_tensor.unsqueeze(0))
  activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)
  result = overlay_mask(to_pil_image(back), to_pil_image(activation_map, mode='F'), alpha=0.5)
  result = result.save('/content/expl/expl_{}'.format(image))

In [22]:
score = 0
for image_, mask in zip(sorted(glob.glob('/content/expl/*.jpg')), sorted(glob.glob('{}/*.jpg'.format(masks_dir)))):
  image = cv2.imread(image_)
  image = cv2.resize(image,(112,112))
  blue_img, green_img, red_img = cv2.split(image)
  thresh_mask = cv2.threshold(red_img, 200, 256, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  image = cv2.imread(mask)
  image = cv2.resize(image,(112,112))
  blue_img, green_img, red_img = cv2.split(image)
  thresh = cv2.threshold(red_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
  union = thresh + thresh_mask
  intersection = thresh * thresh_mask
  union_count = np.count_nonzero(union)
  intersection_count = np.count_nonzero(intersection)
  score += intersection_count/ union_count

In [23]:
score/len(images)

0.08481985620368636

In [ ]:
new_score = 0
for orig, image_, mask in zip(sorted(glob.glob("{}/*.jpg".format(original_dir))) ,sorted(glob.glob('/content/expl/*.jpg')), sorted(glob.glob('{}/*.jpg'.format(masks_dir)))):
  im = cv2.imread(orig)
  im = im[:, :, [2, 1, 0]]
  bb, bb_iou = bb_from_image(image_)
  bb = np.array(bb)
  Y = create_mask(bb, im)
  bb_exp, bb_exp_iou = bb_from_image(mask)
  bb_exp = np.array(bb_exp)
  Y = create_mask(bb_exp, im)
  new_score += get_iou(bb_exp, bb)

In [ ]:
new_score/len(images)

0.03977321416271284